In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin
import numpy as np

In [2]:
link_list=[]
base_url = "https://www.ad.co.il"
url = "https://www.ad.co.il/nadlanrent?sp275=17413&sp277=17557"
response = requests.get(url)
if response.status_code == 200:
    print("Success")
else:
    print("Failure")
    
soup = BeautifulSoup(response.content, 'html.parser')

divs = soup.find_all('div', class_='card-body p-md-3')

for div in divs:
    link_tag = div.find('a', href=True)
    if link_tag:
        full_url = urljoin(base_url, link_tag['href'])
        link_list.append(full_url)
link_list=link_list[:-3]

Success


In [3]:
url = "https://www.ad.co.il/nadlanrent?sp275=17413&sp277=18321"
response = requests.get(url)
if response.status_code == 200:
    print("Success")
else:
    print("Failure")
    
soup = BeautifulSoup(response.content, 'html.parser')

divs = soup.find_all('div', class_='card-body p-md-3')

for div in divs:
    link_tag = div.find('a', href=True)
    if link_tag:
        full_url = urljoin(base_url, link_tag['href'])  
        link_list.append(full_url)
link_list=link_list[:-3]

Success


In [4]:
link_list[0:5]

['https://www.ad.co.il/ad/16213740',
 'https://www.ad.co.il/ad/16184049',
 'https://www.ad.co.il/ad/16061374',
 'https://www.ad.co.il/ad/16021171',
 'https://www.ad.co.il/ad/16206458']

In [5]:
len(link_list)

56

In [6]:
apartments = []

fields_map = {
    'פרטי הנכס': 'property_type',
    'שכונה': 'neighborhood',
    'כתובת': 'address',
    'חדרים': 'room_num',
    'קומה': 'floor',
    'שטח בנוי': 'area',
    'שטח גינה': 'garden_area',
    'תאריך כניסה': 'days_to_enter',
    'תשלומים בשנה': 'num_of_payments',
    'ארנונה בחודש': 'monthly_arnona',
    'ועד בית בחודש': 'building_tax',
    'קומות בבניין': 'total_floors'
}

In [7]:
def get_apartment_info(apartment_link):
    apartment_dict = {key: np.nan for key in fields_map.values()}  
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(apartment_link, headers=headers)
        if response.status_code != 200:
            print("Error: Failed to retrieve page")
            return None

        soup = BeautifulSoup(response.content, 'html.parser')

        table = soup.find('table', class_='table table-sm mb-4')
        if table:
            for row in table.find_all('tr'):
                cols = row.find_all('td')
                if len(cols) < 2:
                    continue

                label = cols[0].get_text(strip=True)
                value = cols[1].get_text(strip=True)

                if label not in fields_map:
                    continue

                key = fields_map[label]

                if label == 'קומה':
                    parts = value.split(' מתוך ')
                    apartment_dict['floor'] = parts[0].strip()
                    if len(parts) == 2:
                        apartment_dict['total_floors'] = parts[1].strip()
                elif label == 'שטח גינה':
                    apartment_dict['garden_area'] = value if value else 0
                elif label == 'תאריך כניסה':
                    apartment_dict['days_to_enter'] = 0 if value == "מיידית" else np.nan
                else:
                    apartment_dict[key] = value

        desc = soup.find('p', class_='text-word-break')
        if desc:
            apartment_dict['description'] = desc.get_text(separator=' ', strip=True)

        h2s = soup.select('div.d-flex.justify-content-between h2')
        if len(h2s) > 1:
            apartment_dict['price'] = h2s[1].get_text(strip=True)

        features = ["חניה", "מחסן", "מעלית", "מזגן", "נגישות", "סורגים", 'ממ"ד', "מרפסת", "מרוהטת", "משופצת"]
        for div in soup.select('div.card-icon.col-6.d-inline, div.card-icon.col-6.d-inline.disabled'):
            span = div.find('span', class_='px-1')
            if span:
                text = span.get_text(strip=True)
                if text in features:
                    apartment_dict[text] = 0 if 'disabled' in div.get('class', []) else 1

        gallery = soup.find('div', class_='col-12 d-flex mt-3 justify-content-center flex-wrap')
        apartment_dict['num_of_images'] = len(
            gallery.find_all('div', class_='justify-content-center px-1')) if gallery else 0

        apartments.append(apartment_dict)

    except Exception as e:
        print(f"שגיאה: {e}")
        return None

In [8]:
for apartment_link in link_list:
    get_apartment_info(apartment_link)
df = pd.DataFrame(apartments)
df

,property_type,neighborhood,address,room_num,floor,area,garden_area,days_to_enter,num_of_payments,monthly_arnona,...,מזגן,חניה,"ממ""ד",מרפסת,נגישות,סורגים,מעלית,מחסן,משופצת,num_of_images
0,גג/פנטהאוז,הצפון הישן החלק הדרום מזרחי,ישראליס 4,3.5,4,100,NaN,0.0,12,500,...,1,0,0,1,0,0,1,0,1,8
1,דירה,הצפון הישן החלק הדרום מזרחי,ריינס 24,3,קרקע,120,15,NaN,12,600,...,0,0,0,1,0,0,0,0,0,12
2,דירה,הצפון הישן החלק הדרום מזרחי,רוזנבאום 8,3.5,1,100,NaN,0.0,12,700,...,1,1,0,1,0,0,0,0,0,7
3,דירה,הצפון הישן החלק הדרום מזרחי,יוסף אליהו 11,3,1,90,NaN,0.0,12,300,...,1,0,0,1,0,0,0,0,0,8
4,גג/פנטהאוז,הצפון הישן החלק הדרום מזרחי,הירשנברג 3,2,4,45,NaN,0.0,12,450,...,1,0,0,1,0,0,0,0,0,7
5,גג/פנטהאוז,הצפון הישן החלק הדרום מזרחי,המלך ג'ורג' 103,2,4,40,65,0.0,12,NaN,...,1,0,0,1,0,0,0,0,0,13
6,גג/פנטהאוז,הצפון הישן החלק הדרום מזרחי,בר כוכבא 46,1,4,24,30,0.0,12,250,...,1,0,0,1,0,0,0,0,1,8
7,דירה,הצפון הישן החלק הדרום מזרחי,"ש""ץ 15",3,2,48,NaN,NaN,12,298,...,0,0,0,0,0,0,0,0,0,1
8,דירה,הצפון הישן החלק הדרום מזרחי,האלקושי 4,2.5,1,50,NaN,0.0,12,309,...,1,0,0,1,0,1,0,0,1,4
9,דירה,הצפון הישן החלק הדרום מזרחי,"בשדרות ח""ן 5",3,5,85,NaN,0.0,12,500,...,1,0,0,0,0,0,0,0,1,7


<div dir="rtl" style="text-align: right; font-family: Arial; font-size: 16px; line-height: 1.6;">

במהלך הניתוח שמנו לב שבחלק מהדירות מופיע הערך "קרקע" (כלומר 0) בעמודת total_floors, בעוד שבעמודת floor מופיע ערך גבוה מ-0 (למשל 3, 4 וכו'). מצב זה אינו הגיוני, שכן לא ייתכן שדירה נמצאת בקומה שלישית בבניין שמוגדר כ"קרקע" בלבד.

כדי לטפל בחוסר התאמה זה, ביצענו את הפעולות הבאות:

1. המרה של ערכי "קרקע" לערך מספרי 0 בעמודות floor ו-total_floors.
2. תיקון מקרים שבהם הקומה (floor) גבוהה ממספר הקומות הכולל (total_floors) על ידי עדכון total_floors כך שיהיה לפחות שווה לקומה של הדירה.

שיטה זו מבטיחה שנשמור על עקביות לוגית בין נתוני הקומה לבין מספר הקומות הכולל של הבניין, וכך נוכל להפיק תובנות אמינות יותר מהנתונים.

</div>


In [9]:
df['floor'] = df['floor'].replace('קרקע', 0)
df['total_floors'] = df['total_floors'].replace('קרקע', 0)

df['floor'] = pd.to_numeric(df['floor'], errors='coerce')
df['total_floors'] = pd.to_numeric(df['total_floors'], errors='coerce')

df.loc[df['floor'] > df['total_floors'], 'total_floors'] = df['floor']

In [10]:
display(df.loc[[46], ['monthly_arnona', 'building_tax']])

,monthly_arnona,building_tax
46,10101010,777777777


<div dir="rtl" style="text-align: right; font-family: Arial; font-size: 16px; line-height: 1.6;">

במהלך בדיקת הנתונים זיהינו מספר דירה בה הערכים בעמודות <code>monthly_arnona</code> ו-<code>building_tax</code> היו חריגים במיוחד – ערכים מספריים עצומים שלא משקפים הגיון כלכלי או מציאותי. נראה כי הערכים הללו הוזנו באופן שגוי או שרירותי.

כדי לטפל במקרה זה , בחרנו גישה מבוססת הקשר: לקחנו את כל הדירות באותה שכונה של הדירה הבעייתיות , תוך התעלמות מהדירה הבעייתית עצמה, וחישבנו את ממוצע הערכים בעמודות <code>monthly_arnona</code> ו-<code>building_tax</code>.

את הערכים החריגים החלפנו בערכים הממוצעים שחושבו, וכך שמרנו על עקביות והיגיון בנתונים, תוך צמצום השפעת נתונים שגויים על ניתוחים עתידיים.

</div>


In [11]:
df['monthly_arnona'] = pd.to_numeric(df['monthly_arnona'], errors='coerce')
df['building_tax'] = pd.to_numeric(df['building_tax'], errors='coerce')

ramat_aviv_g = df[(df['neighborhood'] == 'רמת אביב ג') & (df.index != 46)]

mean_arnona = ramat_aviv_g['monthly_arnona'].mean()
mean_building_tax = ramat_aviv_g['building_tax'].mean()

df.at[46, 'monthly_arnona'] = mean_arnona
df.at[46, 'building_tax'] = mean_building_tax

In [12]:
display(df.loc[[46], ['monthly_arnona', 'building_tax']])


,monthly_arnona,building_tax
46,743.5,384.222222


<div dir="rtl" style="text-align: right; font-family: Arial; font-size: 16px; line-height: 1.6;">

במהלך עיבוד הנתונים זיהינו כי במספר דירות מופיעים ערכי <code>NaN</code> בעמודת <code>address</code>, כלומר כתובת חסרה. מאחר ושאיבת נתוני מרחק מכיכר דיזנגוף התבצעה באמצעות Google Distance Matrix API, היעדר כתובת מנעה מאיתנו לחשב את המרחק עבור אותן דירות.

כדי לטפל בכך, בחרנו להשליך כתובת מייצגת עבור כל שכונה – כתובת שממוקמת במרכז השכונה או בנקודה מרכזית בה. את הכתובות הללו אספנו בצורה ידנית באמצעות Google Maps, תוך הקפדה שהן ישקפו מיקום מייצג ככל הניתן עבור הדירות החסרות.

שיטה זו אפשרה לנו להשלים את נתוני המרחקים בצורה סבירה, תוך שמירה על רלוונטיות גיאוגרפית גבוהה.

</div>


In [13]:
df.at[11, 'address'] = 'ריינס 30'
df.at[48, 'address'] = 'מאיר פיינשטיין 2'

In [14]:
dizengoff_square = "Dizengoff Square, Tel Aviv, Israel"

def get_distance_from_center(address):
    if pd.isna(address) or isinstance(address, float):
        return "לא זמין"
    
    if "תל אביב" not in address:
        address = f"{address}, Tel Aviv, Israel"
    
    api_key = 'here put your api key'  # יש להזין את הAPI KEY שלך!
    
    if api_key == 'here put your api key':
        return "API Key לא הוגדר"
    
    url = "https://maps.googleapis.com/maps/api/distancematrix/json"
    params = {
        'origins': address,  
        'destinations': dizengoff_square,  
        'key': api_key,  
        'mode': 'driving' 
    }
    
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if data['rows'][0]['elements'][0]['status'] == 'OK':
            distance = data['rows'][0]['elements'][0]['distance']['text']
            return distance
        else:
            return "לא זמין"
    else:
        return "שגיאה בבקשה"

df['distance_from_center'] = df['address'].apply(get_distance_from_center)

df

,property_type,neighborhood,address,room_num,floor,area,garden_area,days_to_enter,num_of_payments,monthly_arnona,...,חניה,"ממ""ד",מרפסת,נגישות,סורגים,מעלית,מחסן,משופצת,num_of_images,distance_from_center
0,גג/פנטהאוז,הצפון הישן החלק הדרום מזרחי,ישראליס 4,3.5,4,100,NaN,0.0,12,500.0,...,0,0,1,0,0,1,0,1,8,1.3 km
1,דירה,הצפון הישן החלק הדרום מזרחי,ריינס 24,3,0,120,15,NaN,12,600.0,...,0,0,1,0,0,0,0,0,12,2.1 km
2,דירה,הצפון הישן החלק הדרום מזרחי,רוזנבאום 8,3.5,1,100,NaN,0.0,12,700.0,...,1,0,1,0,0,0,0,0,7,1.3 km
3,דירה,הצפון הישן החלק הדרום מזרחי,יוסף אליהו 11,3,1,90,NaN,0.0,12,300.0,...,0,0,1,0,0,0,0,0,8,1.2 km
4,גג/פנטהאוז,הצפון הישן החלק הדרום מזרחי,הירשנברג 3,2,4,45,NaN,0.0,12,450.0,...,0,0,1,0,0,0,0,0,7,1.5 km
5,גג/פנטהאוז,הצפון הישן החלק הדרום מזרחי,המלך ג'ורג' 103,2,4,40,65,0.0,12,NaN,...,0,0,1,0,0,0,0,0,13,0.7 km
6,גג/פנטהאוז,הצפון הישן החלק הדרום מזרחי,בר כוכבא 46,1,4,24,30,0.0,12,250.0,...,0,0,1,0,0,0,0,1,8,0.7 km
7,דירה,הצפון הישן החלק הדרום מזרחי,"ש""ץ 15",3,2,48,NaN,NaN,12,298.0,...,0,0,0,0,0,0,0,0,1,1.7 km
8,דירה,הצפון הישן החלק הדרום מזרחי,האלקושי 4,2.5,1,50,NaN,0.0,12,309.0,...,0,0,1,0,1,0,0,1,4,1.2 km
9,דירה,הצפון הישן החלק הדרום מזרחי,"בשדרות ח""ן 5",3,5,85,NaN,0.0,12,500.0,...,0,0,0,0,0,0,0,1,7,0.9 km


In [15]:
df = df.rename(columns={
    'מרוהטת': 'is_furnished',
    'מזגן': 'ac',
    'חניה': 'has_parking',
    'ממ"ד': 'has_safe_room',
    'מרפסת': 'has_balcon',
    'נגישות': 'handicap',
    'סורגים': 'has_bars',
    'מעלית': 'elevator',
    'מחסן': 'has_stotsge',
    'משופצת': 'is_renovated'
})

In [16]:
new_order = [
    'property_type', 'neighborhood', 'address', 'room_num', 'floor', 
    'area', 'garden_area', 'days_to_enter', 'num_of_payments', 
    'monthly_arnona', 'building_tax', 'total_floors', 'description', 
    'has_parking', 'has_stotsge', 'elevator', 'ac', 'handicap', 
    'has_bars', 'has_safe_room', 'has_balcon', 'is_furnished', 
    'is_renovated', 'price', 'num_of_images', 'distance_from_center'
]

df = df[new_order]

df = df.replace('מיידית', 0)

df['price'] = (
    df['price']
    .astype(str)  
    .str.replace(r'[^\d.]', '', regex=True)  
    .replace('', np.nan)  
    .astype(float)  
)

df['distance_from_center'] = (
    df['distance_from_center']
    .astype(str)
    .str.replace(r'[^\d.]', '', regex=True)
    .replace('', np.nan)
    .astype(float)
)

In [17]:
df['garden_area'] = df['garden_area'].fillna(0)

dtypes = {
    'property_type': 'string',
    'neighborhood': 'string',
    'address': 'string',
    'room_num': 'float',
    'floor': 'int',
    'area': 'int',
    'garden_area': 'int',
    'days_to_enter': 'int',
    'num_of_payments': 'int',
    'monthly_arnona': 'int',
    'building_tax': 'int',
    'total_floors': 'int',
    'description': 'string',
    'has_parking': 'int',
    'has_stotsge': 'int',
    'elevator': 'int',
    'ac': 'int',
    'handicap': 'int',
    'has_bars': 'int',
    'has_safe_room': 'int',
    'has_balcon': 'int',
    'is_furnished': 'int',
    'is_renovated': 'int',
    'price': 'float',
    'num_of_images': 'int',
    'distance_from_center': 'float'
}

for col, dtype in dtypes.items():
    if col in df.columns:
        if dtype in ['int', 'float']:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            if dtype == 'int':
                df[col] = df[col].round()
                df[col] = df[col].astype('Int64')
        else:
            df[col] = df[col].astype(dtype)

In [18]:
df.dtypes

property_type           string[python]
neighborhood            string[python]
address                 string[python]
room_num                       float64
floor                            Int64
area                             Int64
garden_area                      Int64
days_to_enter                    Int64
num_of_payments                  Int64
monthly_arnona                   Int64
building_tax                     Int64
total_floors                     Int64
description             string[python]
has_parking                      Int64
has_stotsge                      Int64
elevator                         Int64
ac                               Int64
handicap                         Int64
has_bars                         Int64
has_safe_room                    Int64
has_balcon                       Int64
is_furnished                     Int64
is_renovated                     Int64
price                          float64
num_of_images                    Int64
distance_from_center     

In [19]:
df.to_csv('C:/Users/97252/Desktop/degree/שנה ג/סמסטר ב/AdvancedDA/פרוייקט חלק א/filtered_data.csv', index=False, encoding='utf-8-sig')

In [20]:
df

,property_type,neighborhood,address,room_num,floor,area,garden_area,days_to_enter,num_of_payments,monthly_arnona,...,ac,handicap,has_bars,has_safe_room,has_balcon,is_furnished,is_renovated,price,num_of_images,distance_from_center
0,גג/פנטהאוז,הצפון הישן החלק הדרום מזרחי,ישראליס 4,3.5,4,100,0,0,12,500,...,1,0,0,0,1,0,1,15000.0,8,1.3
1,דירה,הצפון הישן החלק הדרום מזרחי,ריינס 24,3.0,0,120,15,<NA>,12,600,...,0,0,0,0,1,0,0,8850.0,12,2.1
2,דירה,הצפון הישן החלק הדרום מזרחי,רוזנבאום 8,3.5,1,100,0,0,12,700,...,1,0,0,0,1,0,0,87000.0,7,1.3
3,דירה,הצפון הישן החלק הדרום מזרחי,יוסף אליהו 11,3.0,1,90,0,0,12,300,...,1,0,0,0,1,0,0,9400.0,8,1.2
4,גג/פנטהאוז,הצפון הישן החלק הדרום מזרחי,הירשנברג 3,2.0,4,45,0,0,12,450,...,1,0,0,0,1,0,0,6500.0,7,1.5
5,גג/פנטהאוז,הצפון הישן החלק הדרום מזרחי,המלך ג'ורג' 103,2.0,4,40,65,0,12,<NA>,...,1,0,0,0,1,0,0,6000.0,13,0.7
6,גג/פנטהאוז,הצפון הישן החלק הדרום מזרחי,בר כוכבא 46,1.0,4,24,30,0,12,250,...,1,0,0,0,1,0,1,5500.0,8,0.7
7,דירה,הצפון הישן החלק הדרום מזרחי,"ש""ץ 15",3.0,2,48,0,<NA>,12,298,...,0,0,0,0,0,0,0,7800.0,1,1.7
8,דירה,הצפון הישן החלק הדרום מזרחי,האלקושי 4,2.5,1,50,0,0,12,309,...,1,0,1,0,1,0,1,7200.0,4,1.2
9,דירה,הצפון הישן החלק הדרום מזרחי,"בשדרות ח""ן 5",3.0,5,85,0,0,12,500,...,1,0,0,0,0,0,1,8500.0,7,0.9
